TODO: Replace computational complexity with computational intensity

Multicore processors dominate the commercial marketplace, with the consequence that almost all computers are now parallel computers. From a developer's perspective, a core issue is to develop algorithms such that available compute resources are used optimally and in a purposeful way. But measuring and reporting performance of a computer program is often challenging because of non-deterministic nature of the execution times. This is because of the increasing complexity of the underlying architecture and unavoidable performance impacts of operating system. Therefore, summarizing the performance of an algorithm to facilitate insightful comparisions with other algorithms forms the central goal of performance modelling in High Performance Computing.  

### Changes in the distribution of execution time as computational intensity increases

To understand the non deterministic characteristics of execution times (as a result of system noise), we do repeated executions of an algorithm, keeping most of the variational factors (architecture, OS and run time settings of the computer program) fixed. At this point, let's respresent an algorithm by some abstract amount of work done by the underlying processor. For now, let us restrict our scope of algorithms to square matrix matrix multiplications. DGEMM is the implementation for double precision matrix matrix multiplication. A general matrix matrix multiplication involves multiplication of two matrices of sizes mxn and nxk. For a square matrix m=n=k. This size of the matrix is proportional to the amount of work done by the underlying processor. Therefore, every algorithm in our following discussion would represent a matrix matrix multiplication of a specific size which abstracts the amount of work done my the machine. Thus, the X-axis in Fig[2] would represent an ordering of increasing computational intensity. 


<img src="noise.png" />

As done in [1], minimum time **t_min** is assumed to be close to noiseless execution and all other times **ti**, perturbed by **t_i - t_min** are shown in Fig 1. The experiments were done on a MacBook Pro - CoffeeLake with 8 threads, utilizing Quadcore CPU @ 2.30GHz with turboboost switched off. Fig [1] shows the histogram and log-histogram of noise from specific cases at two ends of the intensity spectrum - algorithms DGEMM 200 and DEGEMM 1000. The characteristics of the distribution of execution times changes as computational intensity increases and the following properties are observed : 

1. The statistical dispersion (spread) of the distrubition depends on the intensity with which the underlying processor is operating. This indicates the heteroscedastic condition. The spread is less at the lower end (DGEMM 200 ; red histogram) than at the higher end (DEGEMM 1000). The evolution of distributions of execution time is shown in Figure 2.

2. Firgure 3 shows the plot of intensity vs computational rate. Computational rate is defined as the number of floating point operations computed per second. Since we know the FLOPs computed by DGEMM, it is possible to compute the computational rate from execution time. It can be seen that the co-efficient of variation (ratio of the spread by mean) is not constant, and it is somewhat larger around the ridge point, (where there is a transition from memory bound to compute bound) than at the ends.

3. Multi-modal nature of log distribution (blue histogram) is often observed at the lower end. This is because, for some of the executions, data is in the cache, thereby resulting in lesser execution time. Therefore, same intensity can have multiple performance levels depnding on weather or not, the data is in cache. It is often difficult to capture the impacts of cache in performance modelling. But the multi-modal log distributions at the lower end and it's gradual transition to the skewed gaussian captures the effects of the cache. Another reason for multi-modality is the presence of low frequency noise (observed as blunt spikes in the scatter plot of DGEMM 1000) resulting in a small bump at the right end log histogram.   


<img src="heteroS.png" width=500/>

### Performance modelling

Having seen that the performance metrics can be a distribution and that the characteristics of these distribution can depend on the computational intensity, we need metrics that provide summary statistics about the range of predictions as opposed to only point estimates. Perhaps several quantiles, say 0.05, 0.5 (median) and 0.95 quantiles together can be a better descriptor for a distribution. Therefore, quantile based regression techniques[?] are prefered over classical regression methods. The abstract optimization problem is 


Choosing L as the quantile loss [?], 

it would be possible to get a fit over the required quantile. This loss function tries to give different penalties to overestimation and underestimation based on the value of chosen quantile (gamma). For example, specifying gamma as 0.05, 0.5 and 0.95, we can get the fit of 5th quantile, median and the 95th quantile respectively (see fig [?]). 

When L is least square loss, f(x) approximates the mean of each distribution. However, prediction interval from least square regression is based on an assumption that residuals (y — y_hat) have constant variance across the intensity spectrum, which is violated because of heteroscedasity condition. Therefore y is replaced with sample mean y = sum m .. (which satisfies the homoscedacity condition as a consequence of central limit theorem) in the optimization problem 1. But, as our distributions are skewed to the left, the mean estimate becomes sensitive and biased towards the higher quantiles and hence it is not clear if it can be a good statistic. Therefore, it has been quite common to do a regression fit with y as meadian / minimum (Repeat an algorithm M times and get the min / median) optimized for ad-hoc solutions. With least square loss, we also do not know if the homoscedacity condition would be satisfied for min / median. (This approximation of min / median can be different from the quantile approximation) 


### Research Questions

The Evolution of DGEMM distribution along the intensity spectrum contains a lot of information about the behaviour of the underlying processor. It would be an interesting application if we can predict the performance of some black box algorithm Z whose features (FLOP counts, intensity etc) are not known, by running it several times and relate the distributions of Z to the distributions of DEGEMM and there by make an approximate prediction of the underlying resource usage (say X-Axis in fig [?]). 

It is often difficult to get the exact computational intensity of an algorithm even if we have the source code and know it's FLOP counts. In Fig[?], we used the matrix size as an indicator for the computational intensity for the specific kernel DGEMM and not the actual intensity. But in reality, there are several number of kernels. Therefore, when the distributions are obtained for a different kernel, the indicator of intensity for that kernel may not relate to the indicator of intensity for DGEMM. However, fig [?] by itself captures the behaviour of the underlying processor. Thus, when both the kernels are run on the same processor under similar machine settings (like number of threads), we can expect some mapping of distributions between kernels and it would be interesting if we can align the intensity indicators of different kernels to some reference kernel (say DGEMM).   

If it is not possible to relate the performance models of different kernels with respect to a base kernel, we may have to get the performance model separately for each kernel. But getting the data in fig[?] for all the kernels can be time consuming. In such a case, it would be useful to know a good estimate for sample size for each distribution (no. of repetitions for each algorithm) to obtain a decent performance model.  

#### new : 

Given that we know certain quantiles of two algorithms X and Y. How do we compare them and rank them? What is the number of repetioions required to make a good comparision.


### Extra 

Quantile

These summary statistics are often mean, median, minimum or other quantiles. Which statistics would be a good summary and would facilitate insightful comparision is still a topic of debate. Let (y_x)i be the i-th  measurement of execution time of an algorithm with input feature x (in our context, it is the matrix size) and f(x) be the regression fit which represents one of the summary statistics. Then the optimization problem would be     

minimize $L(\tilde{y} -f) + \lambda R_f$

Such that 

> $\tilde{y}_x = f_q(x) + \epsilon_x \\ \\
   x \in {1,...N} \text{ where N is the total number of measurements} \\
   \tilde{y} \text{  is the measurement} \\
   f_q(x) \text{  is the best fit} \\
   \epsilon \text{ is the error} \\
   \text{Loss function L and Regularizer R are suitably chosen such that the expected error over all data points can be minimized}$

Depending on the choice of Loss function L, different summary statistics can be approximated. 

Choosing L as absolute difference, mnedian can be approximated

Choosing L as the least square error and R as Tikonov regularization, we end up with regularized least square optimization. But Least square approximation expects homoscedacity condition and therefore it cannot be directly applied with execution time. Therefore, sample mean is computed for every algorithm y_tilde = sum of y by M



Choosing L as the least square error and R as Tikonov regularization, we end up with regularized least square optimization. But Least square approximation expects homoscedacity condition and therefore it cannot be directly applied with execution time. Therefore, it is more common to fit estimates for each algorithm (like mean, median). That is, repeat each algorithm M times and compute the mean / median. Then solve the regression problem to compute the mean / median (in this case, y_tilde would be mean / median). When doing so, we are left with an obvious question - What would be a good choice for the sample size M.

Choosing L as quantile loss - we directly fit the execution time -- now we dont face with this question of choosing M



Now that we have a bunch of data points and their execution times, we want to solve the following optimization problem

  problem

  quantiles

Now that there are a set of distributions, we want to use it get the quantiles for arbitary computational complexity (matrix size) using quantile regression. 

How many times should I repeat each algorithm to get the quantile fit?

1. Given some features of the algorithm (FLOP counts, Memory reference, kernel library etc), can we predict the characteristic properties of the underlying distribution of execution time?


2. When the features of the algorithm (FLOP counts, Memory references) are not available, is it possible to use the observed distribution (by running algorithm several times) and relate it to one of the distributions of DEGEMM and there by make an approximate prediction of the underlying resource usage (X-Axis)


Let Program Z be the algorithm for which we want to estimate certain performance metric. Consider the following two cases:

1. Program Z is a black box. That is, we do not have access to the algorithm specific features ('matrix size' in our context. Usually it is FLOP counts, number of memory references etc.)

2. We have access to algorithm specific features.

**Case 1**:

When the features of the algorithm (FLOP counts, Memory references) are not available, is it possible to use the observed distribution (by running algorithm several times) and relate it to one of the distributions of DEGEMM and there by make an approximate prediction of the underlying resource usage (X-Axis)
